# Neural ODE & SDE models
---

Цели:
- Познакомиться с Нейронными ОДУ
- Познакомиться с Нуйронными СДУ
- Узнать, где их можно использовать

Содержание:
- Neural ODE
    - Введение

Ссылки:
1. [Neural Ordinary Differential Equations](https://arxiv.org/pdf/1806.07366.pdf)
2. [Знакомство с Neural ODE](https://habr.com/ru/companies/ods/articles/442002/)
3. [Efficient and Accurate Gradients for Neural SDEs](https://arxiv.org/pdf/2105.13493.pdf)

Используемые пакеты:
1. [Репо torchdiffeq](https://github.com/rtqichen/torchdiffeq/)
2. [Репо torchsde](https://github.com/google-research/torchsde/)

## Neural ODE

### Введение

В 2019 году вышла статья **Neural Ordinary Differential Equations** и взяла какие-то (уточнить какие) награды на конференциях. В чем была основная идея и какие проблемы решала данная статья?

Рассмотрим простую рекурентную сеть, которая генерирует дискретные временные последовательности
$$
h_{t+1} = h_t + f(h_t, \theta_t)
$$
где $t=0,\dots,T$ — переменная дискретного времени и $h_t\in\mathbb{R}^D$ — состояния модели на $t$-ом шаге.

Вопрос: Что будет, если мы увеличим число слоев и уменьшим размер шага по времени? В пределе получим, что данное представление соответствует ОДУ первого порядка
$$
\dfrac{dh(t)}{dt} = f(h(t), t, \theta)
$$
зная значение функции $h(t)$ в начальный момент времени $h(0)$, мы можем получить его значение $h(T)$ решив задачу Коши
$$
\begin{cases}
\dfrac{dh(t)}{dt} = f(h(t), t, \theta)\\
h(0) = h_0
\end{cases}
$$
решить которую мы можем численно. 

При этом по авторы статьи выделяют следующие плюсы
- **Эффективность по памяти:** Авторы предлагают более эфективный способ рассчета обратного распространения ошибки нежели стандартный.

- **Адаптивные вычисления:** Варьируя методы решения задачи Коши ОДУ можно балансировать между сложностьи вычислений и невязкой (ошибкой метода).

- **Скалируемые и инвертируемые нормализующие потоки:** Авторы показывают, как можно упростить модель NF [рассмотрим в следующей лекции].

- **Непрерывные во времени модели временных динамик:** Очевидно, что благодаря подходу можно обучать сети, моделирующие непрерывные динамические системы.

### Методы решения ОДУ

Рассмотрим задачу Коши первого порядка обыкновенного дифференциального уравнения
$$
\begin{cases}
\dfrac{dz(t)}{dt} = f(z(t), t) \\
z(t_0) = z_0
\end{cases}
$$

Такое уравнение мы можем решать численно методами Рунге-Кутты. Для того чтобы понять, как это работает, рассмотрим вывод самого простого метода — метода Эйлера. Проинтегрируем обе части уравнения
$$
\dfrac{dz(t)}{dt} = f(z(t), t)⇒ dz(t) = f(z(t), t)dt⇒\int dz(t) = \int f(z(t), t)dt
$$
Так как численное решение не может быть рассчитано на всем бесконечном временном отрезке, возьмем отрезок $[t_0, t_N]$ для которого определен отрезок $[z_0, z_N]$. Тогда решение на этом отрезке расписывается как определенный интеграл
$$
\int\limits_{z_0}^{z_N} dz(t) = \int\limits_{t_0}^{t_N} f(z(t), t)dt⇒z_N - z_0 = \int\limits_{t_0}^{t_N} f(z(t), t)dt⇒z_N = z_0 + \int\limits_{t_0}^{t_N} f(z(t), t)dt
$$
Для численного интегрирования воспользуемся методом прямоугольников. Для этого введем равномерную сетку по времени количеством значений $N$ и с шагом $h$ такие, что $h = \frac{|t_1 - t_0|}{N}↔N=\frac{|t_1 - t_0|}{h}$. Так сетка по времени выглядит как $t_i=t_0+h\cdot i,~i=0,\dots,N$.
$$
\int\limits_{t_0}^{t_N} f(z(t), t)dt≈∑\limits_{i=0}^N hf(z_i, t_i)⇒z_N = z_0 + ∑\limits_{i=0}^N hf(z_i, t_i)
$$
Но так как в задаче Коши нам дано только значение $z(t_0)$, то решать задачу будем итерационно
$$
z_i = z_{i-1} + hf(z_{i-1}, t_{i-1})
$$

Оценим его точность. Для этого разложим аналитическое решение в точке $t_i + h$ в ряд Тейлора
$$
z(t_{i}) = z(t_{i-1} + h) = z(t_{i-1}) + hz'_t(t_{i-1})+O(h^2) = z(t_{i-1}) + hf(z_{i-1}, t_{i-1})+O(h^2)
$$
И вычтем решение методом Эйлера из аналитического решения
$$
\Delta = z(t_{i-1}) + hf(z_{i-1}, t_{i-1}) + O(h^2) - z_{i-1} + hf(z_{i-1}, t_{i-1}) = O(h^2)
$$
Получили ошибку на одном шаге. Очевидно, что для получения ошибки на всем интервале, нужно проитегрировать 